In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tabulate import tabulate
from IPython.display import display, HTML
from google.colab import files
import io
import math
import random
import itertools

In [2]:
def upload_df(dataset_name):

    uploaded = files.upload()
    df = pd.read_csv(io.BytesIO(uploaded[dataset_name]))
    try: 
        df.drop('Unnamed: 0', axis=1, inplace=True)
    except:
        pass
    display(HTML(df.head().to_html()))
    return df

In [3]:
def save_df(df, file_name):

    df.to_csv(file_name)
    files.download(file_name)

In [6]:
class ClusteringPreparations:

    def __init__(self, df):
        self.df = df
        self.c_df = None

    def most_appear_hotels(self, num):
        most_appear_h = self.df.groupby(['Hotel Name']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(num)
        display(HTML(most_appear_h.head().to_html()))
        # taking only the 150 most-appearing hotel rows in the original df
        self.df = self.df[self.df['Hotel Name'].isin(most_appear_h['Hotel Name'])]

    def most_appear_checkins(self, num):
        most_appear_c = self.df.groupby(['Checkin Date']).size().reset_index(name='counts').sort_values('counts', ascending=False).head(num)
        display(HTML(most_appear_c.head().to_html()))
        # now in the original df take only rows that are in the 40 most appear Checkin Date
        self.df = self.df[self.df['Checkin Date'].isin(most_appear_c['Checkin Date'])]

    def create_clustering_df(self):
        dates = self.df['Checkin Date']
        dates = list(dict.fromkeys(dates))
        # mapping for each date concat each discount code list of list [in each one date & 4 the codes 1,2,3,4]
        tmp = list(map(lambda date: list(map(lambda code: str(code) + " " + date  , [1,2,3,4])), dates))
        # make tmp one list.
        merged = list(itertools.chain(*tmp))
        # create new data frame, we will work on it.
        df_fw =  pd.DataFrame(columns = ['Hotel Name'] + merged)
        for hotel, hotels in self.df.groupby(['Hotel Name']):
            # new row for df_fw
            row = {'Hotel Name': hotel}
            # group by combination of both columns: 'Checkin Date','Discount Code' (key), 
            # then for each combination key take the min price from the entries
            for key,entries in hotels.groupby(['Checkin Date','Discount Code']):
                lowest_price = entries.loc[entries['Discount Price'].idxmin(axis=1)]['Discount Price']
                row[str(int(key[1])) + " " + key[0]] = lowest_price
            df_fw = df_fw.append(row, ignore_index=True)        
        df_fw = df_fw.fillna(-1)
        self.c_df = df_fw
        display(HTML(self.c_df.head().to_html()))

    def normalize_clustering_df(self):
        # normalize each row:
        for i, row in self.c_df.drop(columns = ['Hotel Name']).iterrows():  
            # Init max and min
            min_price = max_price = row.max()
            # Finds minimum value
            for col in self.c_df.drop(columns = ['Hotel Name']):
                if row[col] != -1 and row[col] < min_price:
                    min_price = row[col]
            # Normalize row cells
            self.c_df.iloc[i, 1:161] = self.c_df.iloc[i ,1:161].apply(lambda price: -1 if price == -1 else self.normalize(min_price, max_price, price))
        display(HTML(self.c_df.head().to_html()))
    
    @staticmethod
    def normalize(min_price, max_price, price):
        # Normalize price in range between 0:100
        return 100 if min_price == max_price else (price - min_price) / (max_price - min_price) * 100 
    
    def get_old_df(self):
        return self.df

    def get_new_df(self):
        return self.c_df

In [5]:
# upload dataset
df = upload_df('Hotels_data_cleaned.csv')

Saving Hotels_data_cleaned.csv to Hotels_data_cleaned.csv


,Snapshot ID,Snapshot Date,Checkin Date,Days,Original Price,Discount Price,Discount Code,Available Rooms,Hotel Name,Hotel Stars,DayDiff,WeekDay,DiscountDiff,DiscountPerc
0,1,2015-07-17,2015-08-12,5,1178,6.946976,1,6,Best Western Plus Seaport Inn Downtown,3,26,Wednesday,4.927254,2.460851
1,1,2015-07-17,2015-08-19,5,1113,6.889591,1,8,Best Western Plus Seaport Inn Downtown,3,33,Wednesday,4.875197,2.465553
2,1,2015-07-17,2015-08-13,5,4370,8.352319,1,3,The Peninsula New York,5,27,Thursday,4.867534,1.090186
3,1,2015-07-17,2015-07-26,5,1739,7.418781,1,18,Eventi Hotel a Kimpton Hotel,4,9,Sunday,4.276666,1.420771
4,1,2015-07-17,2015-08-12,5,1739,7.421776,1,3,Eventi Hotel a Kimpton Hotel,4,26,Wednesday,4.204693,1.348797


In [7]:
clustering_preparations = ClusteringPreparations(df)
# find out the 150 most appear hotels
clustering_preparations.most_appear_hotels(150)

,Hotel Name,counts
327,Newark Liberty International Airport Marriott,5346
194,Hilton Garden Inn Times Square,4892
374,Residence Inn Newark Elizabeth Liberty International Airport,4314
538,Westin New York at Times Square,3792
290,Loews Regency New York Hotel,3613


In [8]:
# find out the 40 most appear Checkin Date
clustering_preparations.most_appear_checkins(40)

,Checkin Date,counts
116,2015-11-11,2302
88,2015-10-14,1887
109,2015-11-04,1884
32,2015-08-19,1883
102,2015-10-28,1861


In [9]:
# create a new dataframe with the prices for every check-in date for each hotel and for each discount code
clustering_preparations.create_clustering_df()

,Hotel Name,1 2015-08-13,2 2015-08-13,3 2015-08-13,4 2015-08-13,1 2015-08-12,2 2015-08-12,3 2015-08-12,4 2015-08-12,1 2015-08-19,2 2015-08-19,3 2015-08-19,4 2015-08-19,1 2015-09-16,2 2015-09-16,3 2015-09-16,4 2015-09-16,1 2015-09-17,2 2015-09-17,3 2015-09-17,4 2015-09-17,1 2015-08-26,2 2015-08-26,3 2015-08-26,4 2015-08-26,1 2015-09-11,2 2015-09-11,3 2015-09-11,4 2015-09-11,1 2015-09-18,2 2015-09-18,3 2015-09-18,4 2015-09-18,1 2015-08-27,2 2015-08-27,3 2015-08-27,4 2015-08-27,1 2015-09-09,2 2015-09-09,3 2015-09-09,4 2015-09-09,1 2015-08-28,2 2015-08-28,3 2015-08-28,4 2015-08-28,1 2015-09-10,2 2015-09-10,3 2015-09-10,4 2015-09-10,1 2015-09-30,2 2015-09-30,3 2015-09-30,4 2015-09-30,1 2015-10-01,2 2015-10-01,3 2015-10-01,4 2015-10-01,1 2015-10-02,2 2015-10-02,3 2015-10-02,4 2015-10-02,1 2015-10-07,2 2015-10-07,3 2015-10-07,4 2015-10-07,1 2015-10-14,2 2015-10-14,3 2015-10-14,4 2015-10-14,1 2015-10-15,2 2015-10-15,3 2015-10-15,4 2015-10-15,1 2015-10-16,2 2015-10-16,3 2015-10-16,4 2015-10-16,1 2015-10-21,2 2015-10-21,3 2015-10-21,4 2015-10-21,1 2015-10-22,2 2015-10-22,3 2015-10-22,4 2015-10-22,1 2015-10-27,2 2015-10-27,3 2015-10-27,4 2015-10-27,1 2015-10-28,2 2015-10-28,3 2015-10-28,4 2015-10-28,1 2015-10-29,2 2015-10-29,3 2015-10-29,4 2015-10-29,1 2015-10-30,2 2015-10-30,3 2015-10-30,4 2015-10-30,1 2015-11-03,2 2015-11-03,3 2015-11-03,4 2015-11-03,1 2015-11-04,2 2015-11-04,3 2015-11-04,4 2015-11-04,1 2015-11-05,2 2015-11-05,3 2015-11-05,4 2015-11-05,1 2015-11-06,2 2015-11-06,3 2015-11-06,4 2015-11-06,1 2015-11-07,2 2015-11-07,3 2015-11-07,4 2015-11-07,1 2015-11-10,2 2015-11-10,3 2015-11-10,4 2015-11-10,1 2015-11-11,2 2015-11-11,3 2015-11-11,4 2015-11-11,1 2015-11-12,2 2015-11-12,3 2015-11-12,4 2015-11-12,1 2015-11-13,2 2015-11-13,3 2015-11-13,4 2015-11-13,1 2015-11-18,2 2015-11-18,3 2015-11-18,4 2015-11-18,1 2015-11-20,2 2015-11-20,3 2015-11-20,4 2015-11-20,1 2015-11-21,2 2015-11-21,3 2015-11-21,4 2015-11-21,1 2015-11-26,2 2015-11-26,3 2015-11-26,4 2015-11-26,1 2015-11-27,2 2015-11-27,3 2015-11-27,4 2015-11-27,1 2015-11-28,2 2015-11-28,3 2015-11-28,4 2015-11-28
0,Aloft Harlem,-1.0,-1.0,-1.0,-1.000000,-1.00000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,6.946976,6.949856,6.895683,6.970730,6.970730,6.970730,6.97073,7.021084,7.059618,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,7.148346,7.189922,7.230563,-1.000000,-1.000000,7.157735,-1.0,7.230563,-1.000000,7.059618,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,7.122867,-1.0,7.069023,-1.000000,7.130099,-1.000000,-1.000000,-1.000000,7.110696,-1.000000,-1.0,7.106606,-1.000000,-1.000000,7.337588,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,7.274480,-1.000000,-1.000000,-1.0,7.008505,-1.000000,-1.000000,-1.000000,7.183112,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.00000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,Andaz 5th Avenue - a concept by Hyatt,-1.0,-1.0,-1.0,7.199678,-1.00000,-1.000000,7.417580,-1.0,-1.0,-1.0,-1.0,7.614805,-1.0,-1.0,7.866722,7.919356,7.657283,7.740230,7.771067,7.96728,-1.0,-1.0,-1.0,-1.0,7.684784,7.830028,7.793587,-1.0,7.588830,7.837949,7.685703,7.613819,-1.000000,-1.0,7.413367,-1.0,-1.0,-1.0,7.984463,-1.0,-1.0,-1.000000,-1.000000,-1.000000,7.760041,-1.000000,-1.000000,-1.00000,-1.000000,7.818028,7.830028,-1.000000,7.689371,7.768533,7.96728,7.928406,7.670895,-1.000000,7.813996,7.7209

In [10]:
# normalize prices in the new df
clustering_preparations.normalize_clustering_df()

,Hotel Name,1 2015-08-13,2 2015-08-13,3 2015-08-13,4 2015-08-13,1 2015-08-12,2 2015-08-12,3 2015-08-12,4 2015-08-12,1 2015-08-19,2 2015-08-19,3 2015-08-19,4 2015-08-19,1 2015-09-16,2 2015-09-16,3 2015-09-16,4 2015-09-16,1 2015-09-17,2 2015-09-17,3 2015-09-17,4 2015-09-17,1 2015-08-26,2 2015-08-26,3 2015-08-26,4 2015-08-26,1 2015-09-11,2 2015-09-11,3 2015-09-11,4 2015-09-11,1 2015-09-18,2 2015-09-18,3 2015-09-18,4 2015-09-18,1 2015-08-27,2 2015-08-27,3 2015-08-27,4 2015-08-27,1 2015-09-09,2 2015-09-09,3 2015-09-09,4 2015-09-09,1 2015-08-28,2 2015-08-28,3 2015-08-28,4 2015-08-28,1 2015-09-10,2 2015-09-10,3 2015-09-10,4 2015-09-10,1 2015-09-30,2 2015-09-30,3 2015-09-30,4 2015-09-30,1 2015-10-01,2 2015-10-01,3 2015-10-01,4 2015-10-01,1 2015-10-02,2 2015-10-02,3 2015-10-02,4 2015-10-02,1 2015-10-07,2 2015-10-07,3 2015-10-07,4 2015-10-07,1 2015-10-14,2 2015-10-14,3 2015-10-14,4 2015-10-14,1 2015-10-15,2 2015-10-15,3 2015-10-15,4 2015-10-15,1 2015-10-16,2 2015-10-16,3 2015-10-16,4 2015-10-16,1 2015-10-21,2 2015-10-21,3 2015-10-21,4 2015-10-21,1 2015-10-22,2 2015-10-22,3 2015-10-22,4 2015-10-22,1 2015-10-27,2 2015-10-27,3 2015-10-27,4 2015-10-27,1 2015-10-28,2 2015-10-28,3 2015-10-28,4 2015-10-28,1 2015-10-29,2 2015-10-29,3 2015-10-29,4 2015-10-29,1 2015-10-30,2 2015-10-30,3 2015-10-30,4 2015-10-30,1 2015-11-03,2 2015-11-03,3 2015-11-03,4 2015-11-03,1 2015-11-04,2 2015-11-04,3 2015-11-04,4 2015-11-04,1 2015-11-05,2 2015-11-05,3 2015-11-05,4 2015-11-05,1 2015-11-06,2 2015-11-06,3 2015-11-06,4 2015-11-06,1 2015-11-07,2 2015-11-07,3 2015-11-07,4 2015-11-07,1 2015-11-10,2 2015-11-10,3 2015-11-10,4 2015-11-10,1 2015-11-11,2 2015-11-11,3 2015-11-11,4 2015-11-11,1 2015-11-12,2 2015-11-12,3 2015-11-12,4 2015-11-12,1 2015-11-13,2 2015-11-13,3 2015-11-13,4 2015-11-13,1 2015-11-18,2 2015-11-18,3 2015-11-18,4 2015-11-18,1 2015-11-20,2 2015-11-20,3 2015-11-20,4 2015-11-20,1 2015-11-21,2 2015-11-21,3 2015-11-21,4 2015-11-21,1 2015-11-26,2 2015-11-26,3 2015-11-26,4 2015-11-26,1 2015-11-27,2 2015-11-27,3 2015-11-27,4 2015-11-27,1 2015-11-28,2 2015-11-28,3 2015-11-28,4 2015-11-28
0,Aloft Harlem,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,11.607311,12.25914,0.0,16.982694,16.982694,16.982694,16.982694,28.377424,37.097320,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,57.175868,66.584321,75.781089,-1.000000,-1.000000,59.300700,-1.0,75.781089,-1.000000,37.097320,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,51.410131,-1.0,39.225786,-1.000000,53.046648,-1.000000,-1.000000,-1.000000,48.656024,-1.000000,-1.0,47.730489,-1.0,-1.000000,100.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.0,85.719062,-1.000000,-1.000000,-1.0,25.530934,-1.000000,-1.000000,-1.000000,65.043160,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
1,Andaz 5th Avenue - a concept by Hyatt,-1.0,-1.0,-1.0,0.0,-1.000000,-1.000000,27.657946,-1.0,-1.0,-1.0,-1.0,52.691382,-1.0,-1.0,84.666778,91.347512,58.082972,68.611262,72.525423,97.430425,-1.0,-1.0,-1.0,-1.0,61.573647,80.009244,75.383813,-1.0,49.394357,81.014622,61.690309,52.566145,-1.000000,-1.0,27.123188,-1.0,-1.0,-1.0,99.611377,-1.0,-1.0,-1.000000,-1.00000,-1.0,71.125860,-1.000000,-1.000000,-1.000000,-1.000000,78.486086,80.009244,-1.0,62.155888,72.203814,97.430425,92.496193,59.810725,-1.000000,77.974

In [ ]:
# save the new df
save_df(clustering_preparations.get_new_df(), 'Clustering_df.csv')